- **Which datasets do you anticipate using in your final project? Are there columns that can be used to merge these datasets into a single dataframe?**

The datasets we plan to use in our final project are the section 12 alcohol licenses dataset and crime incidents data report from boston.gov. As well as the BARI census block groups mapping from Harvard dataverse and potentially the 911 calls ecometrics. The columns that can be used to merge these datasets are based on location. The crime and alcohol datasets include the address and street name of the business or incident. They also include latitude and longitude coordinates which are likely going to be most effective to use to find their position on the map. Since the geographic data reported by the Boston police can be slightly off we can't rely too heavily on it's accuracy, therefore when merging the data together with a spatial join it's important to note that these areas of city blocks represent overall crime in this area, but not at any exact address or coordinate. Other shared columns are the dates. The licenses and crime incidents both include the date of issue or occurrence which represents another possible area to merge the data although this is not our main focus for now.


- **What is the unit(s) of analysis you will be studying for your final project? In other words, what is the unit that each row in your final dataframe(s) will represent? Is this the same level of analysis that is represented in your raw data? If not, what columns in your raw data could be used to aggregate it to the level you need for your study?**

The unit of analysis for our project are city issued alcohol licenses in Boston, MA and incidents of alcohol related crimes. This is because we are looking to see if a neighborhood block group with a higher number of alcohol licenses is more prone to alcohol related crimes (Operating Under the Influence, Liquor Violation, and potentially violence related crimes based on peer reviewed research) than neighborhood block groups with fewer total licenses. In the final dataframe we would have latitude and longitude cordinates of these specific filtered crimes which are mapped onto specific city block groups in the greater Boston area. The level of analysis is not the same as represented in the raw data since our raw data includes the licenses and incidents on an individual basis. As stated above, we will need to do some type of aggregation (most likely spatial join) of crimes and licenses based on location from the raw data, in order to evaluate our research question.

### Load in and compile two datasets to find missing items for final question

In [18]:
import pandas as pd

In [19]:
crime_2015 = pd.read_csv('data/crime-incident-reports-2015.csv')
crime_2016 = pd.read_csv('data/crime-incident-reports-2015.csv')
crime_2017 = pd.read_csv('data/crime-incident-reports-2017.csv')
crime_2018 = pd.read_csv('data/crime-incident-reports-2018.csv')
crime = pd.DataFrame()
crime=crime.append(crime_2015, ignore_index=True)
crime=crime.append(crime_2016, ignore_index=True)
crime=crime.append(crime_2017, ignore_index=True)
crime=crime.append(crime_2018, ignore_index=True)

In [20]:
alc = pd.read_csv('data/blb_section_12_alcohol.csv')

In [21]:
colleges = pd.read_csv('data/Colleges_and_Universities.csv')

# start with first crime csv
df_crime = pd.read_csv('data/2012-2015-crime.csv.gz')

# append all crime csv's into one dataframe
for year in range(2015, 2022):
    # form standard name convention for each year
    file_name = 'data/' + str(year) + '-crime.csv.gz'
    df_crime.append(pd.read_csv(file_name), ignore_index=True)

# license data is all in one
df_alc = pd.read_csv('data/alcohol.csv.gz')

df_alc

df_crime

# shows types of criminal offenses, we can use Liquor Violations and Operating Under the Influence
# (or other based on research linking crime and alcohol)
df_crime['OFFENSE_CODE_GROUP'].unique()

### Find percentage of missing data from key rows

print("Percentage of missing 'Lat' data in crime dataset:")
print(sum(df_crime.Lat.isna()) / len(df_crime)*100,'%')

print("Percentage of missing 'Long' data in crime dataset:")
print(sum(df_crime.Long.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'OFFENSE_CODE_GROUP' data in crime dataset:")
print(sum(df_crime.OFFENSE_CODE_GROUP.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'OFFENSE_CODE' data in crime dataset:")
print(sum(df_crime.OFFENSE_CODE.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'OCCURRED_ON_DATE' data in crime dataset:")
print(sum(df_crime.OCCURRED_ON_DATE.isna()) / len(df_crime)*100, '%')

print('-' * 50)

print("Percentage of missing 'gpsx' data in liquor license dataset:")
print(sum(df_alc.gpsx.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'gpsy' data in liquor license dataset:")
print(sum(df_alc.gpsy.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'ISSUED' data in liquor license dataset:")
print(sum(df_alc.issued.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'license_type' data in liquor license dataset:")
print(sum(df_alc.license_type.isna()) / len(df_crime)*100, '%')

- **If you have the data for your project in hand: Evaluate your main columns of interest for missing data. What percentage of rows are missing data? Can you see any patterns in which rows are missing data?**

The main columns of interest chosen in the evaluation above are the location data of both the licenses and crime incidents and various other important factors. The location data for both is split up into latitude and longitude coordinates where some rows have both values NA. In the crime dataset 3.97% of rows contain missing location data and in the licenses dataset 0.09% contained missing location data. Given the crime dataset contains nearly 35x more rows than the licenses, with ~4% of crimes missing a location we are still left with a large number of crime incidents (51,000+) with a location. The other important features for crime including the offense code and occurence date contain no NA values. The issued date column for the licenses contained 0.013% NA values which, similar to the location, is a very small portion of the data. The licenses type column contained 0% NA values.

A general pattern observed in the missing data is that latitude and longitude columns are missing the same data, which is helpful because it means that either the entire location was recorded or not at all. If it was not recorded we can exclude this point. Another note is that in the future we will need additional techniques beyond just checking NA for the location. We could also check if the location is outside of Boston, since we only want to include crimes in this area, and there could be rows with errors in the latitude and longitude that we don't want included in our analysis, but aren't NA.